In [6]:
!pip install -q streamlit
!npm install -g localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 3s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙npm notice
npm notice New major version of npm available! 10.8.2 -> 11.4.1
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.4.1
npm notice To update run: npm install -g npm@11.4.1
npm notice
⠙

In [19]:
%%writefile app.py
import pandas as pd
from transformers import pipeline
import matplotlib.pyplot as plt
import io
import base64
import streamlit as st
from PIL import Image

# --- IMPORTANT: st.set_page_config MUST be the very first Streamlit command for PAGE 1 ---
st.set_page_config(layout="wide", page_title="Student Feedback Analysis")

# Set Matplotlib parameters
plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 16

# --- Data Loading and Model Initialization ---

@st.cache_data
def load_data(file_path, sheet_name):
    try:
        df_loaded = pd.read_excel(file_path, sheet_name=sheet_name)
        # Convert all columns to string to handle mixed types gracefully before filling NaNs
        for col in df_loaded.columns:
            df_loaded[col] = df_loaded[col].astype(str).fillna('')
        return df_loaded
    except FileNotFoundError:
        return None
    except Exception as e:
        return None

@st.cache_resource
def load_models():
    summarizer_hf_model = None
    sentiment_analyzer_model = None
    try:
        summarizer_hf_model = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    except Exception as e:
        pass
    try:
        sentiment_analyzer_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
    except Exception as e:
        pass
    return summarizer_hf_model, sentiment_analyzer_model

file_path = '/content/Feedback on Peer-to-Peer Teaching Methodology (1).xlsx'
sheet_name = 'Sheet1'
student_name_column = 'Name'

df = load_data(file_path, sheet_name)
summarizer_hf, sentiment_analyzer = load_models()

# Define columns that contain actual feedback text, excluding identifiers or irrelevant data
# You might need to refine this list based on your specific survey questions
TEXT_FEEDBACK_COLUMNS = [
    'How effective was the peer-to-peer teaching method in helping you understand the topic?',
    'Compared to traditional faculty lectures, how do you rate peer teaching?',
    'Did peer teaching help you understand the topic better than self-study?',
    'What aspect of peer teaching helped you the most?',
    'Did the peer-led sessions encourage you to actively participate?',
    'How would you rate the clarity of the explanations given by your peers?',
    'Did the presenters use engaging methods to explain concepts?',
    'Which factor do you think affected the clarity of peer teaching the most?',
    'Was the content covered in sufficient depth during peer teaching?',
    'Did peer teaching sessions make complex topics easier to understand?',
    'Did you feel more comfortable asking questions in a peer teaching session than in a faculty-led lecture?',
    'How interactive were the peer teaching sessions?',
    'How often did you participate in discussions during peer teaching?',
    'What aspect of peer teaching made you engage more?',
    'Did presenting in a peer-teaching session improve your confidence?',
    'How did peer teaching help in developing communication skills?',
    'Did you gain any leadership skills through peer teaching?',
    'How clear were the assessment criteria provided for peer teaching evaluation?',
    'Did the assessment method fairly evaluate the performance of the presenters?',
    'Were the weightages assigned to different criteria appropriate?',
    'How well did the "Content Accuracy and Relevance" criterion capture the quality of the presentation?',
    'How effective was the "Clarity of Delivery" criterion in assessing communication skills?',
    'Was the assessment process transparent and unbiased?',
    'Would you suggest any modifications to the assessment method?',
    'What was the biggest challenge you faced in peer teaching?',
    'What would improve future peer-teaching sessions?',
    'Would you recommend continuing peer-to-peer teaching in future classes?',
    'If given the choice, how often would you prefer peer teaching sessions?',
    'How well did your peers handle answering doubts and clarifications?',
    'How would you rate the overall experience of peer teaching?'
]


# --- Core Logic Functions ---

def analyze_sentiment_hf(text):
    if sentiment_analyzer is None:
        return "N/A (Model not loaded)"
    try:
        if not text or text.lower() == 'nan':
            return "N/A"
        result = sentiment_analyzer(text)[0]
        label = result['label']

        if label == 'LABEL_0':
            return 'Negative'
        elif label == 'LABEL_1':
            return 'Neutral'
        elif label == 'LABEL_2':
            return 'Positive'
        else:
            return f"Unknown ({label})"
    except Exception as e:
        return "Error"

def get_student_feedback_and_summary_efficient_hf(student_name_input, dataframe, name_column, summarizer_pipe, sentiment_pipe):
    if dataframe is None or dataframe.empty:
        return """
        <p style="color: #dc3545; text-align: center; margin-top: 20px;">
            <strong>Error: Data not loaded. Please ensure the Excel file is uploaded.</strong>
        </p>
        """, None

    student_row = dataframe[dataframe[name_column].str.strip().str.lower() == student_name_input.strip().lower()]

    if student_row.empty:
        all_students = dataframe[name_column].unique().tolist()
        suggest_students = ", ".join(all_students[:3]) + ("..." if len(all_students) > 3 else "")
        return f"""
        <p style="color: #dc3545; text-align: center; margin-top: 20px;">
            <strong>Student not found.</strong><br>
            Please try one of these: {suggest_students}<br>
            (Names are case-insensitive)
        </p>
        """, None

    feedback_entries_display = [] # For display purposes
    feedback_entries_for_ai = []  # Only text suitable for AI processing

    # Filter columns to only include those relevant for textual feedback and summarization
    relevant_feedback_cols = [col for col in TEXT_FEEDBACK_COLUMNS if col in dataframe.columns]

    if not relevant_feedback_cols:
        return "No relevant feedback columns found in the dataset to analyze for text processing.", None

    for col in relevant_feedback_cols:
        feedback_text = str(student_row[col].iloc[0]).strip()
        if feedback_text and feedback_text.lower() not in ['nan', 'none', '']:
            feedback_entries_display.append(f"- **{col.replace('_', ' ').title()}:** {feedback_text}")
            feedback_entries_for_ai.append(feedback_text) # Only add text for AI

    if not feedback_entries_display:
        return "No valid feedback found for this student.", None

    # Combine feedback for AI. Use a separator that might help the model distinguish different points.
    combined_feedback_for_ai = " ".join(feedback_entries_for_ai)

    summary_text = "Summary not available."
    if summarizer_pipe and combined_feedback_for_ai.strip():
        try:
            # Adjust max_length for potentially longer summaries
            # max_input_length = summarizer_pipe.model.config.max_position_embeddings if hasattr(summarizer_pipe.model.config, 'max_position_embeddings') else 512
            # # Truncate input if too long, leaving space for output tokens
            # if len(combined_feedback_for_ai.split()) > max_input_length - 50:
            #      combined_feedback_for_ai = " ".join(combined_feedback_for_ai.split()[:max_input_length - 50])

            # Increased max_length for summary
            summaries = summarizer_pipe(combined_feedback_for_ai, max_length=200, min_length=50, do_sample=False)
            if summaries and summaries[0]['summary_text'].strip():
                summary_text = summaries[0]['summary_text']
            else:
                summary_text = "Could not generate a concise summary. The feedback might be too short or generic for meaningful summarization."
        except Exception as e:
            summary_text = f"Error generating summary: {e}"
    elif not summarizer_pipe:
        summary_text = "Summarization model not loaded."
    else:
        summary_text = "No sufficient textual feedback provided for summarization."


    overall_sentiment = "Sentiment not available."
    if sentiment_pipe:
        overall_sentiment = analyze_sentiment_hf(combined_feedback_for_ai)

    # Apply custom CSS for overall sentiment and summary sections
    formatted_output = f"""
    <style>
        .sentiment-card, .summary-card {{
            background-color: #f8f9fa; /* Light grey background */
            border-left: 5px solid #957DAD; /* Accent border */
            border-radius: 8px;
            padding: 20px;
            margin-top: 20px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            font-family: 'Arial', sans-serif;
            color: #333333;
        }}
        .sentiment-card h4, .summary-card h4 {{
            color: #6A0DAD; /* Purple header */
            margin-top: 0;
            font-size: 1.5em;
            font-weight: bold;
            margin-bottom: 10px;
        }}
        .sentiment-card p, .summary-card p {{
            font-size: 1.1em;
            line-height: 1.6;
        }}
        .sentiment-positive {{ color: #28a745; font-weight: bold; }}
        .sentiment-negative {{ color: #dc3545; font-weight: bold; }}
        .sentiment-neutral {{ color: #6c757d; font-weight: bold; }}
        .sentiment-error, .sentiment-n/a {{ color: #ffc107; font-weight: bold; }}
    </style>
    <div style="background-color: white; padding: 20px; border-radius: 8px; border: 1px solid #E0BBE4; margin-top: 25px; box-shadow: 0 2px 5px rgba(0,0,0,0.05);">
        <h4>📋 Student Feedback Details</h4>
        <ul>
            {''.join([f'<li>{fb}</li>' for fb in feedback_entries_display])}
        </ul>
    </div>
    <div class="sentiment-card">
        <h4>😊 Overall Sentiment</h4>
        <p class="sentiment-{'positive' if overall_sentiment == 'Positive' else 'negative' if overall_sentiment == 'Negative' else 'neutral' if overall_sentiment == 'Neutral' else 'error'}">{overall_sentiment}</p>
    </div>
    <div class="summary-card">
        <h4>📝 Summary</h4>
        <p>{summary_text}</p>
    </div>
    """
    return formatted_output, student_row.iloc[0]

def ask_student_question_lookup(question_input, student_feedback_series, name_column):
    if student_feedback_series is None or student_feedback_series.empty:
        return "No student feedback loaded or student not found previously. Please analyze a student first."

    # Use only TEXT_FEEDBACK_COLUMNS for Q&A context
    relevant_qa_cols = [col for col in TEXT_FEEDBACK_COLUMNS if col in student_feedback_series.index]
    combined_row_feedback = " ".join([str(student_feedback_series[col]).strip() for col in relevant_qa_cols if str(student_feedback_series[col]).strip().lower() not in ['nan', 'none', '']])

    question_lower = question_input.lower()
    answer = "I can help you understand this student's feedback better. Try asking about their biggest challenge, what they liked, their suggestions, or their overall experience."

    if "challenge" in question_lower or "difficult" in question_lower:
        if "struggled" in combined_row_feedback or "difficult" in combined_row_feedback or "rushed" in combined_row_feedback or "challenge" in combined_row_feedback:
            relevant_answers = [f for f in combined_row_feedback.split('.') if any(keyword in f.lower() for keyword in ['challenge', 'difficult', 'struggled', 'rushed'])]
            answer = f"Based on feedback, challenges mentioned include: '{' '.join(relevant_answers)}'." if relevant_answers else "The feedback doesn't explicitly mention significant challenges."
        else:
             answer = "The feedback doesn't explicitly mention significant challenges."
    elif "like" in question_lower or "positive" in question_lower or "enjoy" in question_lower:
        if "informative" in combined_row_feedback or "helpful" in combined_row_feedback or "rewarding" in combined_row_feedback or "enjoy" in combined_row_feedback:
            relevant_answers = [f for f in combined_row_feedback.split('.') if any(keyword in f.lower() for keyword in ['informative', 'helpful', 'rewarding', 'enjoy', 'liked', 'positive'])]
            answer = f"The student appreciated: '{' '.join(relevant_answers)}'." if relevant_answers else "Specific positive aspects are not prominently highlighted."
        else:
            answer = "Specific positive aspects are not prominently highlighted."
    elif "suggestion" in question_lower or "improve" in question_lower or "better" in question_lower:
        if "wish" in combined_row_feedback or "more" in combined_row_feedback or "needed" in combined_row_feedback or "improve" in combined_row_feedback:
            relevant_answers = [f for f in combined_row_feedback.split('.') if any(keyword in f.lower() for keyword in ['wish', 'more', 'needed', 'improve', 'suggestion'])]
            answer = f"The student suggested: '{' '.join(relevant_answers)}'." if relevant_answers else "No clear improvement suggestions were identified."
        else:
            answer = "No clear improvement suggestions were identified."
    elif "overall experience" in question_lower or "how was" in question_lower:
        overall_sentiment = analyze_sentiment_hf(combined_row_feedback)
        answer = f"The student's overall experience, based on the analysis, was generally **{overall_sentiment}**."

    return answer


def visualize_overall_sentiment_py(dataframe, name_column, sentiment_pipe):
    if dataframe is None or dataframe.empty:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.text(0.5, 0.5, 'No data loaded. Cannot generate chart.', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, color='red')
        ax.axis('off')
        return fig

    if sentiment_pipe is None:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.text(0.5, 0.5, 'Sentiment model not loaded. Cannot generate chart.', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, color='red')
        ax.axis('off')
        return fig

    all_feedback_texts = []
    # Use only TEXT_FEEDBACK_COLUMNS for overall sentiment analysis
    relevant_feedback_cols = [col for col in TEXT_FEEDBACK_COLUMNS if col in dataframe.columns]

    if not relevant_feedback_cols:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.text(0.5, 0.5, 'No relevant feedback columns found for chart.', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, color='red')
        ax.axis('off')
        return fig

    for _, row in dataframe.iterrows():
        combined_row_feedback = " ".join([str(row[col]).strip() for col in relevant_feedback_cols if str(row[col]).strip().lower() not in ['nan', 'none', '']])
        if combined_row_feedback:
            all_feedback_texts.append(combined_row_feedback)

    if not all_feedback_texts:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.text(0.5, 0.5, 'No feedback data available for analysis.', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=12, color='red')
        ax.axis('off')
        return fig

    sentiments = [analyze_sentiment_hf(text) for text in all_feedback_texts]
    sentiment_counts = pd.Series(sentiments).value_counts()

    labels = sentiment_counts.index.tolist()
    sizes = sentiment_counts.values.tolist()
    colors = {
        'Positive': '#28a745',
        'Neutral': '#6c757d',
        'Negative': '#dc3545',
        'Error': '#ffc107',
        'N/A': '#aaaaaa'
    }
    pie_colors = [colors.get(label, '#cccccc') for label in labels]

    fig, ax = plt.subplots(figsize=(8, 8))
    wedges, texts, autotexts = ax.pie(sizes, colors=pie_colors, autopct='%1.1f%%', startangle=90, pctdistance=0.85)

    centre_circle = plt.Circle((0,0), 0.70, fc='white')
    fig.gca().add_artist(centre_circle)

    ax.axis('equal')
    ax.set_title('Overall Sentiment Distribution', color='#6A0DAD', fontsize=16, pad=20)
    ax.legend(wedges, labels, title="Sentiment", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

    plt.setp(autotexts, size=12, weight="bold", color="white")
    plt.setp(texts, size=12, color="#333333")

    plt.tight_layout()

    return fig

# --- Streamlit UI Layout ---
# Custom CSS for the entire app, including the new card designs
st.markdown("""
    <style>
        .reportview-container .main .block-container {
            max-width: 1400px;
            padding-top: 2rem;
            padding-right: 2rem;
            padding-left: 2rem;
            padding-bottom: 2rem;
        }
        h1 {
            text-align: center;
            color: #6A0DAD;
            font-size: 3.5em;
            font-weight: bold;
            text-transform: uppercase;
            margin-bottom: 30px;
            padding-bottom: 10px;
            border-bottom: 4px solid #957DAD;
            letter-spacing: 2px;
        }
        .description {
            text-align: center;
            font-size: 1.1em;
            margin-bottom: 40px;
            color: #333333;
        }
        .stButton>button {
            border-radius: 0px;
            font-size: 1.1em;
            padding: 12px 20px;
            font-weight: bold;
            transition: all 0.2s ease-in-out;
            border: none;
            min-width: 120px;
        }
        .stButton>button:hover {
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
        }
        .primary-button {
            background-color: #957DAD !important;
            color: white !important;
        }
        .primary-button:hover {
            background-color: #6A0DAD !important;
        }
        .secondary-button {
            background-color: #E0BBE4 !important;
            color: #6A0DAD !important;
            border: 1px solid #957DAD !important;
        }
        .secondary-button:hover {
            background-color: #957DAD !important;
            color: white !important;
        }
        .stTextInput>div>div>input {
            border: 2px solid #CCCCCC;
            border-radius: 5px;
            padding: 12px;
        }
        .output-area {
            background-color: white;
            padding: 20px;
            border-radius: 8px;
            border: 1px solid #E0BBE4;
            margin-top: 25px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.05);
            min-height: 200px;
            overflow-y: auto;
        }
        .sentiment-card, .summary-card {
            background-color: #f8f9fa; /* Light grey background */
            border-left: 5px solid #957DAD; /* Accent border */
            border-radius: 8px;
            padding: 20px;
            margin-top: 20px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            font-family: 'Arial', sans-serif;
            color: #333333;
        }
        .sentiment-card h4, .summary-card h4 {
            color: #6A0DAD; /* Purple header */
            margin-top: 0;
            font-size: 1.5em;
            font-weight: bold;
            margin-bottom: 10px;
        }
        .sentiment-card p, .summary-card p {
            font-size: 1.1em;
            line-height: 1.6;
        }
        h3 {
            color: #6A0DAD;
            font-size: 1.8em;
            margin-top: 25px;
            margin-bottom: 20px;
            text-align: left;
            border-bottom: 2px solid #E0BBE4;
            padding-bottom: 5px;
        }
        .stMarkdown ul {
            list-style-type: none;
            padding-left: 0;
        }
        .stMarkdown ul li {
            margin-bottom: 8px;
        }
    </style>
""", unsafe_allow_html=True)

# Define pages
PAGES = {
    "Student Feedback Analysis": "page_1",
    "Overall Sentiment Visualization": "page_2"
}

# --- Page Navigation ---
st.sidebar.title("Navigation")
selection = st.sidebar.radio("Go to", list(PAGES.keys()))
page_id = PAGES[selection]

if page_id == "page_1":
    st.title("Student Feedback Analysis")
    st.markdown("""
        <p class="description">
            This AI-powered tool provides comprehensive insights into student feedback.
            You can analyze individual student responses to get a summary and overall sentiment,
            and ask specific questions about their feedback.
        </p>
    """, unsafe_allow_html=True)

    # --- Check if data and models loaded successfully ---
    if df is None:
        st.error("❌ Data file not found. Please upload 'Feedback on Peer-to-Peer Teaching Methodology (1).xlsx' to /content/ and rerun the app.")
        st.stop()

    if summarizer_hf is None:
        st.warning("⚠️ Summarization model could not be loaded. Summary feature will be unavailable.")
    if sentiment_analyzer is None:
        st.warning("⚠️ Sentiment analysis model could not be loaded. Sentiment features will be unavailable.")

    # --- Initialize session state variables ---
    if 'student_feedback_series' not in st.session_state:
        st.session_state['student_feedback_series'] = None
    if 'analysis_output_html' not in st.session_state:
        st.session_state['analysis_output_html'] = '<p style="color: #666; text-align: center; margin-top: 80px;">Analysis report will appear here.</p>'
    if 'qa_answer_text' not in st.session_state:
        st.session_state['qa_answer_text'] = '<p style="color: #666;">Answer will appear here.</p>'

    col1, col2 = st.columns(2)

    with col1:
        st.markdown("<h3>Enter Student Name to Analyze Feedback</h3>", unsafe_allow_html=True)

        student_name_input = st.text_input("Student Name", placeholder="Type name and press Enter (e.g., Afsheen Zaahrah A)", key="student_name_input")

        feedback_output_placeholder = st.empty()

        if student_name_input:
            with st.spinner("Analyzing feedback..."):
                output_html, student_feedback_data = get_student_feedback_and_summary_efficient_hf(
                    student_name_input, df, student_name_column, summarizer_hf, sentiment_analyzer
                )
                st.session_state.analysis_output_html = output_html
                st.session_state.student_feedback_series = student_feedback_data
                feedback_output_placeholder.markdown(st.session_state.analysis_output_html, unsafe_allow_html=True)
        else:
            st.session_state.analysis_output_html = """
                <div style="text-align: center; margin-top: 60px; color: #666;">
                    <p><strong>Analysis report will appear here.</strong></p>
                    <p style="margin-top: 15px;">
                        Try these sample students:<br>
                        • Afsheen Zaahrah A<br>
                        • John Smith<br>
                        • Sarah Johnson
                    </p>
                </div>
            """
            feedback_output_placeholder.markdown(st.session_state.analysis_output_html, unsafe_allow_html=True)

        if st.button("Clear All", key="clear_all_button", type="secondary"):
            st.session_state['student_name_input'] = ''
            st.session_state.analysis_output_html = '<p style="color: #666; text-align: center; margin-top: 80px;">Analysis report will appear here.</p>'
            st.session_state.qa_answer_text = '<p style="color: #666;">Answer will appear here.</p>'
            st.session_state.student_feedback_series = None
            st.rerun()

    with col2:
        st.markdown("<h3>❓ Ask Questions About This Student's Feedback</h3>", unsafe_allow_html=True)

        if st.session_state.student_feedback_series is not None and not st.session_state.student_feedback_series.empty:
            question_input = st.text_input("Your Question", placeholder="e.g., What was the biggest challenge?", key="question_input")
            ask_button = st.button("Get Answer", key="ask_button", type="primary")

            qa_answer_placeholder = st.empty()
            qa_answer_placeholder.markdown(st.session_state.qa_answer_text, unsafe_allow_html=True)

            if ask_button and question_input:
                with st.spinner("Getting answer..."):
                    answer = ask_student_question_lookup(question_input, st.session_state.student_feedback_series, student_name_column)
                    st.session_state.qa_answer_text = f"""
                        <div style="background-color: #f8f9fa; border-left: 5px solid #E0BBE4; border-radius: 8px; padding: 15px; margin-top: 20px; box-shadow: 0 2px 4px rgba(0,0,0,0.08);">
                            <p style="font-size: 1.1em; line-height: 1.6;"><strong>Answer:</strong> {answer}</p>
                        </div>
                    """
                    qa_answer_placeholder.markdown(st.session_state.qa_answer_text, unsafe_allow_html=True)
        else:
            st.info("Analyze a student first to enable the Q&A section.")
            st.markdown("""
                <div style="background-color: #f8f9fa; border-left: 5px solid #E0BBE4; border-radius: 8px; padding: 15px; margin-top: 20px; box-shadow: 0 2px 4px rgba(0,0,0,0.08);">
                    <p style="font-size: 1.1em; line-height: 1.6;">Answer will appear here.</p>
                </div>
            """, unsafe_allow_html=True)

elif page_id == "page_2":
    # --- IMPORTANT: st.set_page_config for PAGE 2, not needed here as it's already set once
    # This st.set_page_config is for the whole app, so it's placed at the top.

    st.title("Overall Sentiment Visualization")
    st.markdown("""
        <p class="description">
            This page provides a visual overview of the sentiment distribution across all student feedback entries in your dataset.
        </p>
    """, unsafe_allow_html=True)

    if df is None:
        st.error("❌ Data file not found. Cannot generate chart.")
    elif sentiment_analyzer is None:
        st.warning("⚠️ Sentiment analysis model could not be loaded. Cannot generate chart.")
    else:
        st.write("Click the button below to generate a pie chart showing the overall sentiment across all student feedback entries.")

        chart_placeholder = st.empty()
        chart_placeholder.markdown('<p style="color: #666; text-align: center; margin-top: 80px;">Chart will appear here after clicking "Generate Chart"</p>', unsafe_allow_html=True)

        if st.button("Generate Overall Sentiment Chart", key="generate_chart_button_page2", type="primary"):
            with st.spinner("Generating chart..."):
                fig = visualize_overall_sentiment_py(df, student_name_column, sentiment_analyzer)
                chart_placeholder.pyplot(fig)
                plt.close(fig)

        st.markdown("""
            <p style="margin-top: 20px; font-style: italic; color: #666;">
                *Note: This chart analyzes the sentiment of ALL individual feedback responses across the entire dataset.*
            </p>
        """, unsafe_allow_html=True)

Overwriting app.py


In [ ]:
!nohup streamlit run app.py &

import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!npx localtunnel --port 8501

nohup: appending output to 'nohup.out'
Password/Enpoint IP for localtunnel is: 34.29.172.35
⠙⠹⠸⠼⠴⠦⠧your url is: https://lazy-papayas-throw.loca.lt
